In [1]:
import pandas as pd
import numpy as np

### dataディレクトリの中に格納されているxlsxファイルのパスを取得

In [2]:
import glob

file_list = glob.glob('data/fille_*.xlsx')

file_list

['data\\fille_1706.xlsx', 'data\\fille_1709.xlsx', 'data\\fille_1712.xlsx']

### データフレームとして読み込みリストに格納

In [3]:
dtypes={'科目コード':str, '金額':int}
parse_date = ['決算日']

#df = pd.read_excel('data/fille_1712.xlsx',dtype=dtypes, parse_dates=parse_date)

df_list = [pd.read_excel(file ,dtype=dtypes, parse_dates=parse_date) for file in file_list]

display(df_list[0], df_list[1], df_list[2])

,決算日,科目コード,金額
0,2017-06-30,10001,10
1,2017-06-30,10002,20
2,2017-06-30,10003,30
3,2017-06-30,10002,40
4,2017-06-30,10001,50


,決算日,科目コード,金額
0,2017-09-30,10001,100
1,2017-09-30,10002,200
2,2017-09-30,10003,300
3,2017-09-30,10002,400
4,2017-09-30,10001,500


,決算日,科目コード,金額
0,2017-12-31,10001,10000
1,2017-12-31,10002,20000
2,2017-12-31,10003,30000
3,2017-12-31,10002,40000
4,2017-12-31,10001,50000


In [4]:
# 読み込んだデータフレームの各カラムのデータ型の確認
for df in df_list:
    display(df.dtypes)

決算日      datetime64[ns]
科目コード            object
金額                int32
dtype: object

決算日      datetime64[ns]
科目コード            object
金額                int32
dtype: object

決算日      datetime64[ns]
科目コード            object
金額                int32
dtype: object

## SQLserverへ接続開始する関数

In [5]:
def connect_to_sqlserver(database, driver='{SQL Server}', server = 'localhost\SQLEXPRESS',trusted_connection='yes'):
    '''
    関数内容
    ・PythonからSQLserverへ接続する関数
    Input
    ・database：使用するデータベース名(予めSQLserver上で「CREATE DATABASE [データベース名]」というコードを実行させて作成しておく)
    関数使用方法
    ・connect_to_sqlserver('C10000_database')
    '''
    import pyodbc
    conn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';DATABASE='+database+';Trusted_Connection='+trusted_connection+';')
    curs = conn.cursor()
    
    return conn, curs

### SQLserverへ接続開始する関数を実行

In [6]:
conn, curs = connect_to_sqlserver('C10000_database')

import pyodbc

driver='{SQL Server}'
server = 'localhost\SQLEXPRESS' 
database = 'C10000_database' 
trusted_connection='yes'

conn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';DATABASE='+database+';Trusted_Connection='+trusted_connection+';')
curs = conn.cursor()

## データベース内にテーブルを作成する関数
参考：https://itsakura.com/python-sqlite-insert

In [7]:
def create_tables_in_sqlserver(file_name_list, curs=curs, conn=conn):
    '''
    関数内容
    ・データベース内にテーブルを作成する関数(カラムは[決算日]、[科目コード]、[金額]で決め打ち)
    Input
    ・file_name_list：読み込んだ全ファイル名を格納したリスト
    関数使用方法
    ・create_tables(file_name_list)
    '''

    for file_name in file_name_list:
        curs.execute(
        'CREATE TABLE ' + file_name +\
        '(決算日 DATE,科目コード NVARCHAR(10),金額 NUMERIC(25))')

        conn.commit()

In [8]:
# ディレクトリに格納されているファイルの名前のみ取得
import os

file_name_list = [os.path.splitext(os.path.basename(file))[0] for file in file_list]

file_name_list

['fille_1706', 'fille_1709', 'fille_1712']

### データベース内にテーブルを作成する関数を実行

In [9]:
create_tables_in_sqlserver(file_name_list)

## データベース内のテーブルにデータを追加する関数

In [10]:
def insert_into_sqlserver(file_name_list, df_list, curs=curs, conn=conn):
    '''
    関数内容
    ・データベース内のテーブルにデータを追加する関数
    Input
    ・file_name_list：読み込んだ全ファイル名を格納したリスト
    ・df_list：読み込んだ全データフレームを格納したリスト
    関数使用方法
    ・insert_into_sqlserver(file_name_list, df_list)
    '''
    for file_name, df in zip(file_name_list, df_list):
        values_tuple = [tuple(row) for row in df.values]
        # カラムが複数ある結果、挿入したい値が複数ある場合はcurs.executemanyを使う
        curs.executemany(
            'INSERT INTO ' + file_name +'(決算日, 科目コード, 金額) VALUES (?,?,?)',values_tuple
        )

        conn.commit()  

### データベース内のテーブルにデータを追加する関数を実行

In [11]:
insert_into_sqlserver(file_name_list, df_list)

## SQLserverへの接続を切る関数

In [12]:
def disconnect_from_sqlserver(curs=curs, conn=conn):
    '''
    関数内容
    ・SQLserverへの接続を切る関数
    '''
    curs.close()
    conn.close()

### SQLserverへの接続を切る関数を実行

In [13]:
disconnect_from_sqlserver()

curs.close()
conn.close()

参考

https://qiita.com/gaborotta/items/3f2f2fd492163a1ec007

https://qiita.com/YoshitakaOkada/items/d6d97c5954adae148b7a